In [19]:
## Google colab
#from google.colab import drive
#drive.mount('/content/drive')
#import os
#os.chdir("/content/drive/MyDrive/Kaggle_competition")
##os.listdir()

In [20]:
import numpy as np
import pandas as pd

In [21]:
import sys 
sys.path.append("src")
import src.support as sp

In [22]:
df = pd.read_csv("data/train.csv", index_col=0)
df.head(2)

,carat,cut,color,clarity,depth,table,x,y,z,price
id,,,,,,,,,,
0,0.30,Premium,D,SI2,62.4,58.0,4.31,4.28,2.68,6.353
1,1.01,Ideal,E,VVS2,62.7,56.0,6.42,6.46,4.04,9.183


## 0. Gestión de outliers

Tras el análisis realizado en el EDA se puede apreciar que la columna carat explica de manera significativa el precio, y crece hasta llegar al valor 1.96. A partir de ahí encontramos los precios más elevados y aumenta la importancia del las variables color, clarity y cut para explicarlos. Por este motivo la gestión de outliers consiste en dividir el df en ese punto y realizar el entrenamiento por separado.

In [ ]:
bins = [0,1.96,10]  
labels = ['df_1', 'df_2']  
df['part_df'] = pd.cut(df['carat'], bins=bins, labels=labels)

df_1 = df[df['part_df'] == 'df_1']
df_2 = df[df['part_df'] == 'df_2']
df_1 = df_1.drop(['x', 'y', 'z','depth','table','part_df'], axis=1)
df_2 = df_2.drop(['x', 'y', 'z','depth','table','part_df'], axis=1)
df_1= pd.DataFrame(df_1).reset_index()
df_2= pd.DataFrame(df_2).reset_index()


## 1. Encoding

Todas las variables no numéricas tienen orden, por lo que utilizaremos ordinal encoder sobre las columnas color, clarity y cut.

In [23]:
orden_color = ['J', 'I', 'H', 'G', 'F', 'D', 'E']
orden_clarity = ['SI2', 'I1', 'SI1', 'VS2', 'VS1', 'VVS2', 'IF', 'VVS1']
orden_cut = ['Fair', 'Premium', 'Good', 'Very Good', 'Ideal']

df = sp.Preprocesado(df).ordinal_encoder( "color", orden_color)
df = sp.Preprocesado(df).ordinal_encoder( "clarity", orden_clarity)
df = sp.Preprocesado(df).ordinal_encoder( "cut", orden_cut)

df = df.drop(['color', 'clarity', 'cut','x', 'y', 'z'], axis=1)

In [24]:
df.head(2)


,carat,depth,table,price,color_oe,clarity_oe,cut_oe
id,,,,,,,
0,0.30,62.4,58.0,6.353,5,0,1
1,1.01,62.7,56.0,9.183,6,5,4


## 2. Estandarización

In [25]:
df= sp.Preprocesado(df).robust_scaler(['carat', 'depth', 'table'])

In [37]:
df.head(2)

,carat,depth,table,price,color_oe,clarity_oe,cut_oe
id,,,,,,,
0,-0.625000,0.4,0.333333,6.353,5,0,1
1,0.484375,0.6,-0.333333,9.183,6,5,4


## 3. Predicción

#### 3.1. Decision Tree

In [27]:
resultados_dt= sp.Prediccion(df).decision_tree_training('price', 150, 150)

DecisionTreeRegressor(max_depth=29, max_features=2, min_samples_leaf=25,
                      min_samples_split=50)


In [28]:
resultados_dt

,MAE,MSE,RMSE,R2,set,modelo
0,0.121742,0.027063,0.164509,0.973500,test,Decission Tree
1,0.116538,0.024787,0.157440,0.976098,train,Decission Tree


#### 3.2. Random Forest

In [32]:
resultados_rf = sp.Prediccion(df).random_forest_training('price', 150, 150)

RandomForestRegressor(max_depth=19, max_features=2, min_samples_leaf=25,
                      min_samples_split=25)


In [33]:
resultados_rf

,MAE,MSE,RMSE,R2,set,modelo
0,0.104897,0.019299,0.138921,0.981103,test,Random Forest
1,0.101900,0.018443,0.135806,0.982216,train,Random Forest


#### 3.3. Gradient Boosting

In [34]:
resultados_gb = sp.Prediccion(df).gradient_training('price', 150, 150)

Fitting 10 folds for each of 1700 candidates, totalling 17000 fits
GradientBoostingRegressor(max_depth=16, max_features=2, min_samples_leaf=25,
                          min_samples_split=75)


In [35]:
resultados_gb

,MAE,MSE,RMSE,R2,set,modelo
0,0.078052,0.010447,0.102212,0.989770,test,Random Forest
1,0.062653,0.006852,0.082776,0.993393,train,Random Forest
